# Génération de données synthétiques

In [1]:
import numpy as np
import pandas as pd
from math import exp, log, log10, sqrt
from scipy.integrate import odeint
from scipy.stats import norm, lognorm

# The Complete model
def deriv(y, t, phiS, phiL, deltaS, deltaL, deltaAb):
    dydt = phiS * exp(-deltaS * t) + phiL * exp(-deltaL * t) - deltaAb * y
    return dydt

def analytic(A0, time, phiS, phiL, deltaS, deltaL, deltaAb):
    y = []
    for t in time:
        A=(A0-phiS/(deltaAb-deltaS)-phiL/(deltaAb-deltaL))*exp(-deltaAb*t)\
        +phiS/(deltaAb-deltaS)*exp(-deltaS*t)+phiL/(deltaAb-deltaL)*exp(-deltaL*t)
        y.append(A)
    return y

def sample_id_params(pop_params,groupHav720 = False):
    # sample parameters from their distributions
    A0 = norm.rvs(model_params['A0_mean'],model_params['A0_std'])
    phiS = exp(norm.rvs(model_params['ln_phiS_mean'],model_params['ln_phiS_std']))
    deltaAb = exp(norm.rvs(model_params['ln_deltaAb_mean'],model_params['ln_deltaAb_std']))
    
    if groupHav720:    
        phiL = exp(norm.rvs(model_params['ln_phiL_mean'],model_params['ln_phiL_std'])+
                   model_params['beta_phiL_Hav720'])
        deltaS = exp(norm.rvs(model_params['ln_deltaS_mean'],model_params['ln_deltaS_std'])+
                   model_params['beta_deltaS_Hav720'])
        deltaL = exp(norm.rvs(model_params['ln_deltaL_mean'],model_params['ln_deltaL_std'])+
                   model_params['beta_deltaL_Hav720'])
    else:
        phiL = exp(norm.rvs(model_params['ln_phiL_mean'],model_params['ln_phiL_std']))
        deltaS = exp(norm.rvs(model_params['ln_deltaS_mean'],model_params['ln_deltaS_std']))
        deltaL = exp(norm.rvs(model_params['ln_deltaL_mean'],model_params['ln_deltaL_std']))
        
    return A0, (phiS, phiL, deltaS, deltaL, deltaAb)

In [2]:
# True parameters: we suppose that they are log-normal distributed
ln_phiS_mean = log(1)
ln_phiS_std = 0.2

ln_phiL_mean = log(0.54)
ln_phiL_std = 0.1

ln_deltaS_mean = log(0.069)
ln_deltaS_std = 0.5

ln_deltaL_mean = log(1.8e-6)
ln_deltaL_std = 1

ln_deltaAb_mean = log(0.79)
ln_deltaAb_std = 0.1

beta_phiL_Hav720 = -1
beta_deltaS_Hav720 = -0.5
beta_deltaL_Hav720 = 3

# Initial conditions on A0 is supposed to be normally distributed:
A0_mean = 8
A0_std = 0.1

# Finally, we will add an additive error to log_10 transformed data. The error follows a standard gaussian,
# distribution with variance:
sigma2 = 0.01

model_params = {'ln_phiS_mean':ln_phiS_mean,'ln_phiL_mean':ln_phiL_mean,'ln_deltaS_mean':ln_deltaS_mean,
               'ln_deltaL_mean':ln_deltaL_mean,'ln_deltaAb_mean':ln_deltaAb_mean,
               'ln_phiS_std':ln_phiS_std,'ln_phiL_std':ln_phiL_std,'ln_deltaS_std':ln_deltaS_std,
               'ln_deltaL_std':ln_deltaL_std,'ln_deltaAb_std':ln_deltaAb_std,
               'beta_phiL_Hav720':beta_phiL_Hav720,'beta_deltaS_Hav720':beta_deltaS_Hav720,
               'beta_deltaL_Hav720':beta_deltaL_Hav720,'A0_mean':A0_mean,'A0_std':A0_std}

# Time points: we suppose that all participants have observation at all time points. Note: here time is in months.
time = np.linspace(0,36,10)
    
# We are going to generate 100 patients form HavrixTM 1440 dataset and 100 patients from HavrixTM 720 dataset
N1, N2 = 100, 100

data = []
for n in range(N1+N2):
    if n < N1:
        A0, id_params = sample_id_params(model_params,groupHav720 = False)
        error = norm.rvs(0,sqrt(sigma2))
        phiS, phiL, deltaS, deltaL, deltaAb = id_params
        y_t = analytic(A0, time, phiS, phiL, deltaS, deltaL, deltaAb)
        #ret = odeint(deriv, A0, time, args=id_params)
        #y_t = ret.T[0]
        for t in range(len(y_t)):
            data.append([n+1,time[t],log10(y_t[t])+error,A0,0])
    else:
        A0, id_params = sample_id_params(model_params,groupHav720 = True)
        error = norm.rvs(0,sqrt(sigma2))
        phiS, phiL, deltaS, deltaL, deltaAb = id_params
        y_t = analytic(A0, time, phiS, phiL, deltaS, deltaL, deltaAb)
        #ret = odeint(deriv, A0, time, args=id_params)
        #y_t = ret.T[0]
        for t in range(len(y_t)):
            data.append([n+1,time[t],log10(y_t[t])+error,A0,1])
            
dataframe = pd.DataFrame(data, columns=['ID', 'TIME', 'OBS', 'OBS_0', 'GROUP'])

# Save the obtained dataframe as simulated_AB_response.csv
dataframe.to_csv('simulated_AB_response.csv',sep=',',index=False)

In [ ]:
####if you are using Colab:
from google.colab import files 
files.download('simulated_AB_response.csv')